In [1]:
import polars as pl

In [2]:
t = pl.scan_parquet("./dev-with-negatives.parquet")

In [3]:
t_list = [pl.scan_parquet(f"./dev-with-negative-docs-{i}.parquet") for i in range(1, 101)]

In [ ]:
t = t.with_columns(pl.col("body_length_norm").fill_null(0))
new_t_list = []
for tx in t_list:
    new_t_list.append(tx.with_columns(pl.col("body_length_norm").fill_null(0)))

t = t.with_columns(pl.col("title_length_norm").fill_null(0))
new_new_t_list = []
for tx in new_t_list:
    new_new_t_list.append(tx.with_columns(pl.col("title_length_norm").fill_null(0)))

In [5]:
t_full = t.join(new_new_t_list[0], on="index", suffix="_1")

In [6]:
for i, tx in enumerate(new_new_t_list[1:]):
    t_full = t_full.join(tx, on="index", suffix=f"_{i + 2}")

In [7]:
t_full.sink_parquet("./search_engine/ranking-model/data/dev.parquet")

In [8]:
test = pl.read_parquet("./search_engine/ranking-model/data/dev.parquet")